In [0]:
import numpy as np
import math
import pandas as pd
import time
from google.colab import drive

In [0]:
class VDFT:
    #initialize the class
    def __init__(self, feature_vals, delta, n_min):
        self.feature_vals = feature_vals
        self.n_min = n_min
        self.delta = delta
        self.num_instances_proc = 0
        features = list(feature_vals.keys())
        self.root = tree_node(features)

    def update(self, instance):
        node = self.root.sort_instance(instance)
        node.update_stats(instance)
        self.num_instances_proc += 1
        split_feature = node.splittable(self.delta, self.n_min)
        if (split_feature != None):
            child =  self.node_split(node, split_feature)
            node.add_child(split_feature, child)
            for key in child:
                child[key].parent = node


    def node_split(self, node, split_feature):
        features = node.possible_featuresToSplit
        new_features = [None if f == split_feature else f for f in features]
        child = {}
        for v in self.feature_vals[split_feature]:
            child[v] = tree_node(new_features)
        return(child)
    def accuracy(self, instances):
      true = 0
      for examp in instances:
        leaf = self.root.sort_instance(examp)
        pred = leaf.most_freq()
        if (pred == examp[-1]):
          true = true + 1.
      return(true / len(instances))


In [0]:
class tree_node:
      def __init__(self, possible_featuresToSplit):
        self.class_freq = {} #define dictionary for class frequencies
        self.parent = None
        self.child = None
        self.featureToSplit = None
        self.tot_instances = 0
        self.new_instances = 0
        self.possible_featuresToSplit = possible_featuresToSplit
        self.nijk = {i:{} for i in possible_featuresToSplit} #statistics of feature i, value j, class k
      def add_child(self, featureToSplit, child):
        if(child):
          self.child = child
          self.featureToSplit = featureToSplit
          self.nijk.clear()
        else:
            print('the child is empty')

      def most_freq(self):
        if (self.class_freq):
            predict = max(self.class_freq, key=self.class_freq.get)
        else:
            class_freq = self.parent.class_freq
            predict = max(class_freq, key=class_freq.get)
        return(predict)

      def update_stats(self, instance):
        features = self.possible_featuresToSplit
        label = instance[-1]
        for i in features:
            if (i != None):
                val = instance[:-1][features.index(i)] 
                if (val not in self.nijk[i]):  # if the value does not exist
                    d = {label : 1}
                    self.nijk[i][val] = d
                else:
                    if (label not in self.nijk[i][val]): #if the label does not exist
                        self.nijk[i][val][label] = 1
                    else:
                        self.nijk[i][val][label] += 1
        self.new_instances += 1
        self.tot_instances += 1
        if (label not in self.class_freq):
            self.class_freq[label] = 1
        else:
            self.class_freq[label] += 1


      def sort_instance(self, instance):
        val = 0
        if (self.child != None):
            indx = self.possible_featuresToSplit.index(self.featureToSplit)
            val = instance[:-1][indx]
            #print('sort ex val',self.class_freq)
            return(self.child[val].sort_instance(instance))
        else:
            return(self)
      
      def entropy(self, class_freqs):
        tot_instances = 0
        for k in class_freqs:
            tot_instances += class_freqs[k]
        if (tot_instances == 0):
            return(0) 
        entropy = 0
        for k in class_freqs:
            if(class_freqs[k] != 0):
                entropy += -(class_freqs[k] / float(tot_instances)) * math.log2(class_freqs[k] / float(tot_instances))
            else:
                entropy += 0
        return(entropy)


      def information_gain(self, featID):
        njk = self.nijk[featID]
        class_frequency = self.class_freq

        total_instances = self.tot_instances
        if (total_instances == 0):
            return(0)
        old_entropy = self.entropy(class_frequency)

        new_entropy = 0
        for j in njk:
            count = 0
            for k in njk[j]:
                count += njk[j][k]
            new_entropy += (count/float(total_instances)) * (self.entropy(njk[j]))

        info_gain = old_entropy - new_entropy
        return info_gain


      def splittable(self, delta, n_min):
        if(self.new_instances < n_min):
            return(None)
        else:
            self.new_instances = 0  # reset the count
        max_split_feature = 0
        second_max_split_feature = 0
        X_a = ''
        for feature in self.possible_featuresToSplit:
            if (feature != None):
                val = self.information_gain(feature)
                if (val > max_split_feature):
                    max_split_feature = val
                    X_a = feature
                elif (val < max_split_feature and val > second_max_split_feature):
                    second_max_split_feature = val
        R = math.log10(len(self.class_freq))
        sigma = np.sqrt((R**2) * np.log(1/delta) / (2 * n))
        if (max_split_feature - second_max_split_feature > sigma):
            return(X_a)
        else:
            return(None)
   

In [46]:
drive.mount('/content/drive')
start_time = time.time()
df = pd.read_csv('/content/drive/My Drive/UCI_Credit_Card.csv', header=0)
df.describe()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [47]:
df.drop(['ID'], inplace=True, axis =1)
titles = list(df.columns.values)
features = titles[:-1]
feature_values = {f:None for f in features}

for f in features:
  if (df[f].dtype == np.float64 or df[f].dtype == np.int64):
    df[f] = pd.cut(df[f], bins=3)
    feature_values[f] = df[f].unique()
train_size=24000
array = df.head(train_size).values
set1 = []
set2 = []
set3 = []
set4=  []
possible_features_to_split = titles[:-1]
cnt = 0
for i in range(len(array)):
  cnt += 1
  if (cnt <= 10000):
    set1.append(array[i])
  elif (cnt > 10000 and cnt <= 15000):
    set2.append(array[i])
  elif (cnt > 15000 and cnt <= 20000):
    set3.append(array[i])
  else:
    set4.append(array[i])
    
instances = [set1, set2, set3, set4]
test_size = 6000
test_set = df.tail(test_size).values
testSet = []
for i in range(len(test_set)):
  testSet.append(test_set[i])
num_min_instances=80
delta_prob=0.1
tree = VDFT(feature_values, delta_prob, num_min_instances)
print('Data size: ', len(df))
print('Test size: ', len(test_set))


Data size:  30000
Test size:  6000


In [48]:
n = 0
for training_set in instances:
  n += len(training_set)
  for ex in training_set:
    tree.update(ex)
    #i=i+1
    #if i ==31:
      #raise StopIteration
  print('Training set:', n, end=', ')
  print('ACCURACY:  ',tree.accuracy(testSet))

print("Running time: " ,(time.time() - start_time))

Training set: 10000, ACCURACY:   0.825
Training set: 15000, ACCURACY:   0.8295
Training set: 20000, ACCURACY:   0.831
Training set: 24000, ACCURACY:   0.8303333333333334
Running time:  4.1503167152404785


In [0]:
start_time = time.time()
df = pd.read_csv('/content/drive/My Drive/UCI_Credit_Card.csv', header=0)
df.drop(['ID'], inplace=True, axis =1)

In [50]:
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,30000.0,1,2,2,37,4,3,2,-1,0,0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [0]:
X=df.loc[:, df.columns != 'default.payment.next.month'].head(train_size)

In [0]:
y=df['default.payment.next.month'].astype(float).head(train_size)

In [0]:
#compare it with batch decision tree

from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth= 7, min_samples_leaf= 2, random_state=0)
clf = clf.fit(X, y)

In [0]:
test_set=df.loc[:, df.columns != 'default.payment.next.month'].tail(test_size)
y_test=df['default.payment.next.month'].tail(test_size)

In [43]:
test_set

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
24000,50000.0,1,2,2,23,2,2,0,0,0,0,51246.0,49758.0,48456.0,44116.0,21247.0,20066.0,8.0,2401.0,2254.0,2004.0,704.0,707.0
24001,60000.0,1,2,2,26,0,0,0,0,0,0,58072.0,59040.0,57416.0,55736.0,26958.0,28847.0,2282.0,2324.0,2049.0,2000.0,3000.0,1120.0
24002,400000.0,1,2,2,27,0,0,0,0,0,0,15330.0,8626.0,11470.0,10745.0,20737.0,9545.0,2501.0,10009.0,1437.0,1105.0,510.0,959.0
24003,20000.0,1,5,2,27,5,4,3,2,2,2,21673.0,21051.0,20440.0,19709.0,20113.0,19840.0,0.0,0.0,0.0,900.0,0.0,0.0
24004,50000.0,1,3,2,27,0,0,-2,-2,-1,-1,32590.0,-100.0,0.0,0.0,70.0,120.0,0.0,100.0,0.0,70.0,200.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0
29997,30000.0,1,2,2,37,4,3,2,-1,0,0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0
29998,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0


In [0]:
y_pred=clf.predict(test_set)

In [45]:
from sklearn import metrics 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Running time: " ,(time.time() - start_time))

Accuracy: 0.8293333333333334
Running time:  9.376723051071167
